# Training a Moment Tensor Potential model

In this tutorial, we will see how to train a machine-learning interatomic potential with the MTP model

As for the SNAP example, we need to export the path to the LAMMPS binary with
```bash
export ASE_LAMMPSRUN_COMMAND=/path/to/lmp_serial
```
or directly inside the python script as we will do here

In [1]:
import os

os.environ["ASE_LAMMPSRUN_COMMAND"] = "~/.local/bin/lmp"

Once this is done, we can open the dataset using the io module of ASE

In [2]:
from ase.io import read

configurations = read("../Data/Silicon.traj", index=":")

This dataset correspond to 20 configurations of crystaline Silicon in a 2x2x2 supercell with displacements

In [3]:
print(configurations[0])
print(len(configurations))

Atoms(symbols='Si64', pbc=True, cell=[10.873553019744607, 10.873553019744607, 10.873553019744607], momenta=..., calculator=SinglePointCalculator(...))
20


For the MTP potential, the descriptor and model are handled directly with the the MLIP package.
For this kind of potential, we only need to import the MomentTensorPotential

In [4]:
from mlacs.mlip import MomentTensorPotential

To initialize the descriptor, we need to give it some parameters.

In [5]:
mtp_params = dict(max_dist=4.5,
                  min_dist=1.5,
                  level=4)
fit_params = dict(max_iter=50,
                  bfgs_conv_tol=1e-3)

We also need to define the path to the mtp binary

In [13]:
mlpbin = "/Users/alois/.local/bin/mlp"

We can now initalize our model with this descriptor

In [14]:
mlip = MomentTensorPotential(configurations[0],
                             mlpbin,
                             mtp_parameters=mtp_params,
                             fit_parameters=fit_params,
                             energy_coefficient=1.0,
                             forces_coefficient=1.0,
                             stress_coefficient=1.0)

We can check the parameters of our MLIP

In [15]:
print(repr(mlip))

Moment Tensor Potential
Parameters:
-----------
energy coefficient :    1.0
forces coefficient :    1.0
stress coefficient :    1.0

Descriptor:
-----------
level :                 8
radial basis function : RBChebyshev
Radial basis size :     8
Minimum distance :      1.5
Cutoff :                4.5



To train the model, we need now to add the configurations to the training set.

This is done with the `update_matrices` function of the potential, that takes either an ASE atoms object or a list of atoms.

In [16]:
mlip.update_matrices(configurations)

The model can now be trained using the `train_mlip` function

In [17]:
msg = mlip.train_mlip()
print(msg)

number of configurations for training: 10
number of atomic environments for training: 640
number of configurations for training: 10
number of atomic environments for training: 640
RMSE Energy    0.0018 eV/at
MAE Energy     0.0013 eV/at
RMSE Forces    0.0750 eV/angs
MAE Forces     0.0562 eV/angs
RMSE Stress    0.1406 GPa
MAE Stress     0.1009 GPa




To check the accuracy of our MLIP, we can use the command line ```mlacs correlation``` to plot the correlation between DFT data and MLIP prediction.

In [19]:
%%sh
mlacs correlation MLIP-Energy_comparison.dat --size 10 --datatype energy --save EnergyCorrelation.jpeg --noshow
mlacs correlation MLIP-Forces_comparison.dat --size 10 --datatype forces --save ForcesCorrelation.jpeg --noshow
mlacs correlation MLIP-Stress_comparison.dat --size 10 --datatype stress --save StressCorrelation.jpeg --noshow

<img src="EnergyCorrelation.jpeg" heigh=300 width=300>      <img src="ForcesCorrelation.jpeg" heigh=300 width=300>      <img src="StressCorrelation.jpeg" heigh=300 width=300>

And that's it ! The model is ready to be used and can be found in the Snap directory. The pair_style and pair_coeff needed to use it in LAMMPS can be obtained from the mlip object

In [20]:
print(mlip.pair_style)
print(mlip.pair_coeff)

mlip /Users/alois/Documents/Physique/Work/Develop/mlacs/otf_mlacs/tutorials/Mlip/MomentTensorPotential/MTP/mlip.ini
['* *']


Of course, in real applications the parameters and the size of the dataset will need to be different to obtain an accurate model.